## A Maximum Entropy Algorithm
As derived by Nunez and Llacer (1990) and outlined by Carasso (1999).

This algorithm was derived directly from Bayes' Theorem by maximizing the posterior probability. The prior probability p(f) is given using Maxwell-Boltzmann statistics and the global probability p(g|f) is defined with Poisson noise in the blurred image g. 


Required libraries:

In [1]:
%matplotlib qt
import numpy as np
import hyperspy.api as hs
from ncempy.io import dm
import matplotlib.pyplot as plt
from scipy.signal import peak_widths, find_peaks
import math

Functions:

In [2]:
#convert text files
def txtconverter(numpy_array):
    file = str(numpy_array).replace('[','')
    file = file.replace(']','')
    data = np.fromstring(file, sep=',')
    return data

#sorting data into counts and eV
def find_counts(data):
    counts = data[1:-1:2]
    return counts

def find_ev(data):
    ev = data[0:-1:2]
    return ev

#plot the spectrum with HyperSpy
def hyperspy_plot(ev, counts):
    s = hs.signals.EELSSpectrum(counts)
    s.axes_manager[0].scale = np.diff(ev).mean()
    s.axes_manager[0].unit = 'eV'
    s.axes_manager[0].offset = ev[0]
    s.axes_manager[0].name = 'Energy'
    return s

#FWHM comparisons
def FWHM_testing(sigma, gamma, hs_signal, hs_deconvolved, height):
    
    peaks1, _ = find_peaks(hs_signal, height=1)
    results_half_signal = peak_widths(hs_signal, peaks1, rel_height=0.5)
    
    peaks2, _ = find_peaks(hs_deconvolved, height=height)
    results_half_deconvolved = peak_widths(hs_deconvolved, peaks2, rel_height=0.5)
    
    FWHM_signal = 4 / 1000 * results_half_signal[0]
    FWHM_deconvolved = 4 / 1000 * results_half_deconvolved[0]
    
    Lorentzian_FWHM = 2 * gamma
    Gaussian_FWHM = 2.335 * sigma
    
    relative_error = abs((FWHM_deconvolved[0] - Lorentzian_FWHM)/Lorentzian_FWHM*100)
    
    print("FWHM of signal =", FWHM_signal[0], "eV", 
          "\nFWHM of deconvolved =", FWHM_deconvolved[0], "eV", 
          "\nFWHM of Lorentzian =", Lorentzian_FWHM, "eV", 
          "\nRelative error =",  math.trunc(relative_error), "%\n")
    
#plotting the noise
def noise(Spectrum, deconvolved, PSF):
    noise = np.subtract(Spectrum, np.convolve(deconvolved, PSF, mode='same'))
    return noise

#Richardson-Lucy algorithm (code from Edson Bellido)
def RL(iterations, PSF, Spectrum):
    RL4 = np.copy(Spectrum)
    for i in range(iterations):
        RL1 = np.convolve(PSF, RL4, mode='same')
        RL2 = np.divide(Spectrum,RL1)
        RL3 = np.convolve(PSF, RL2, mode='same')
        RL4 = np.multiply(RL3, RL4)
    return RL4

Load files

In [3]:
#load file as numpy array
Signal = np.loadtxt("D:\Downloads\Signal1.txt",dtype="str")
PSF = np.loadtxt("D:\Downloads\PSF1.txt", dtype='str')
Real = np.loadtxt("D:\Downloads\Real1.txt", dtype='str')

#convert text file to usable numpy array
signal = txtconverter(Signal)
psf = txtconverter(PSF)
real = txtconverter(Real)

#separate data into counts and ev
signal_counts = find_counts(signal)
psf_counts = find_counts(psf)
real_counts = find_counts(real)
ev = find_ev(signal)

### MEM Algorithm

This algorithm contains two adjustable parameters that are used to manipulate the convergence rate and smoothing of the deconvolution. The constant C has a small affect on the convergence rate, but is primarily existent to ensure positivity. The constant rho is known as the "sharpness" parameter, as rho increases, the deconvolution process behaves more like the Richardson-Lucy algorithm, and as rho decreases, the prior probability p(f) becomes dominant, smoothing the high frequency information.

1 <= Rho <= 20 (otherwise behaves like RL)

C >= Rho

In [4]:
def MEM(iterations, PSF, Spectrum):
    rho = 10
    C = 15
    N = np.sum(Spectrum)
    
    MEM = Spectrum
    for i in range(iterations):
        A1 = np.convolve(PSF, MEM, mode='same')
        A2 = np.divide(Spectrum, A1)
        A3 = np.convolve(PSF, A2, mode='same')
        A4 = np.subtract(np.multiply(rho, A3), rho)
        A5 = np.add(np.subtract(A4, np.log10(MEM)), C)
        A6 = N * (np.sum(np.multiply(MEM, A5)))**(-1)
        
        MEM1 = np.convolve(PSF, MEM, mode='same')
        MEM2 = np.divide(Spectrum, MEM1)
        MEM3 = np.convolve(PSF, MEM2, mode='same')
        MEM4 = np.subtract(np.multiply(rho, MEM3), rho)
        MEM5 = np.add(np.subtract(MEM4, np.log10(MEM)), C)
        MEM6 = np.multiply(np.multiply(MEM, MEM5), A6)
        MEM = MEM6
    return MEM

Comparison between MEM and RL:

In [5]:
MEM_deconvolve = MEM(10, psf_counts, signal_counts)
s_MEM = hyperspy_plot(ev, MEM_deconvolve)

RL_deconvolve = RL(10, psf_counts, signal_counts)
s_RL = hyperspy_plot(ev, RL_deconvolve)

s_signal = hyperspy_plot(ev, signal_counts)

In [6]:
print("MEM")
FWHM_MEM = FWHM_testing(0.1, 0.1, s_signal, s_MEM, 0.5)

print("RL")
FWHM_RL = FWHM_testing(0.1, 0.1, s_signal, s_RL, 1)

MEM
FWHM of signal = 0.32627560766739383 eV 
FWHM of deconvolved = 0.26104274337774724 eV 
FWHM of Lorentzian = 0.2 eV 
Relative error = 30 %

RL
FWHM of signal = 0.32627560766739383 eV 
FWHM of deconvolved = 0.23908186476795754 eV 
FWHM of Lorentzian = 0.2 eV 
Relative error = 19 %



In [ ]:
s_MEM.plot()

In [ ]:
s_RL.plot()

In [ ]:
s_signal.plot()